# Getting started with Data Quibbler

Below is a quick introuction tour of *Quibbler* for *Python* (`pyquibbler`). 

You may also consult with the [[Examples]] to quickly demo some of the key functionalities of *Quibbler*.

## Setting up

### Install

To install *Quibbler* use: [Maor]

`pip install pyquibbler`

### Import

`pyquibbler` is conventionally imported as `qb`. In addition, it is convenient to directly import some key functions such as `iquib` and `q` (which will be explained below). Following import, we need to also execute `qb.override_all()` which initiates pyquibbler and configure *NumPy* and *Matplotlib* functions to work with *Quibbler*. Imports of *NumPy* and *Matplotlib*, if needed, shoud follow the execution of `qb.override_all()`. A typical import therefore looks like:

In [2]:
# importing and initializing pyquibbler:
import pyquibbler as qb
from pyquibbler import iquib, q
qb.override_all()

# any other imports:
import matplotlib.pyplot as plt
import numpy as np

### Graphics backend

`pyquibbler` works well with `tk`, `widget` and [Maor].

If you are using *Jupiter Notebook*, use for example:

In [3]:
%matplotlib tk

## The quib object

A `Quib` is an object that represents an output *value* as well as the *function* and *arguments* used to calculate this value.  There are two major types of quibs: *input-quibs* (i-quibs) which take any *Python* object [Maor: true?] as their argument and present it as their value (their function is trivially the identity function), and *function-quibs* (f-quibs) that produce their output value by applying a given function to a given list of arguments, which could be i-quibs, other f-quibs and any additional *Python* objects. 

### Input quib (i-quib)

Any *Python* object can be transformed into an i-quib using the `iquib` command. For example:

In [3]:
my_lucky_number = iquib(7)
my_lucky_number

my_lucky_number = iquib(7)

Note that the string representation of a quib shows its *name* (in this case 'my_lucky_number'; see [[name]] property) and its *function* and *arguments* (in this case, 'iquib(7)'; See [[func]], [[args]], [[kwargs]] properties). 

#### Getting the quib's value using `get_value()`
To get the output *value* of the quib, we use the [[get_value]] method:

In [4]:
my_lucky_number.get_value()

7

#### Input quibs can represent objects of any class [Maor: true?]
Quibs can represent any *Python* object including Numeric, String, List, Tuple, Set, and
Dictionary. They can also represent *NumPy* ndarrays, *Matplotlib* Artists as well as essentially any other type of objects.

For example:

In [5]:
city_data = iquib({'City':'Haifa', 'Population':279247})
city_data

city_data = iquib({'City': 'Haifa', 'Population': 279247})

In [6]:
hello_world = iquib(['Hello','World'])
hello_world.get_value()

['Hello', 'World']

#### Assigning new values to input quibs

Input quibs can be modified by assignments using standard *Python* assignment syntax:

In [7]:
hello_world[0] = 'Hi'
hello_world.get_value()

['Hi', 'World']

To completely replace the value of a quib, even with objects of a different type, use the `assign_value` method:

In [8]:
city_data.assign_value('anything')
city_data.get_value()

'anything'

### Function quib (f-quib)

#### Applying functions or operators to quib arguments creates a *function-quib*
Applying standard functions or operators to quibs, or to combinations of quibs with other objects, creates an *f-quib* whose function is to perform the indicated operation. Quibs can be used as arguments with many standard *Python*, *NumPy* and *Matplotlib* functions ([[Supported function]]). Operators such as `+`, `-`, `/`, `*`, `**`, are also supported. We can therefore easily define a chanined network of functional quibs using standard *Python* syntax.

As a simple example, let's start with an input quib `z` representing a numeric *NumPy* array:

In [9]:
z = iquib(np.array([2, 1, 2, 3]))

We can use this quib in standard functions and operations, just like we would use a normal numeric *NumPy* array. For example:

In [10]:
z_sqr = z ** 2
z_sqr

z_sqr = z ^ 2

The operation above created `z_sqr` which is a functional quib whose _function_ is to square the *value* of `z`.

We can similarly continue with additional downstream operations. Say, calculating the average of the elements of `z_sqr`:

In [11]:
mean_z_sqr = np.average(z_sqr)
mean_z_sqr

mean_z_sqr = average(z_sqr)

#### Quibs are typically defined declaratively (lazy evaluation) 

In general, quib operations are declarative; they define a quib with a specified function and arguments, but do not immediately execute this function. For example, the statement above, `mean_z_zqr = np.average(z_sqr)` creates a new f-quib whose function is to perform `np.average` on the value of `z_sqr`, but this average operation is not yet computed (deferred evaluation). The average function is only evaluated when we ask for the value of the quib.

#### Quib functions are only evaluated when their output value is requested

To calculate the value of a funational-quib, we can use the `get_value()` method:

In [12]:
mean_z_sqr.get_value() # (2^2 + 1^2 + 2^2 + 3^2) / 4 = 4.5

4.5

The statement above cause the evaluation of `mean_z_sqr`, performing the function `np.average` on the *value* of `z_sqr`. This operation, in turn, therefore also caused the evaluation of `z_sqr`. 

#### f-quibs cache their calculated value

Following calculation of its value, a quib can cache the result to avoid unnecessary future calculations. Repeated requests for the value of the quib thereby do not require recalculations. For more about caching, see the [[cache_behavior]] and [[cache_status]] properties. 

### Upstream changes automatically propagate to affect downstream results

When we make changes to a quib, these changes are automatically propagated downstream to affect the values of dependent quibs (recursively). For example, suppose we change one of the elements of our input quib `z`:

In [13]:
z[2] = 0

When such a change is made, downstream dependent quibs are notified that some or all of their cached output is no longer valid (tough, no re-calculation is immediately being performed). Then, when we ask for the value of a downstream quib, it will get recalculated to reflect the upstream change:

In [14]:
mean_z_sqr.get_value() # (2^2 + 1^2 + 0^2 + 3^2) / 4 = 3.5

3.5

### Quib indexing is also interpreted functionally

When we index a quib, we get a new quib whose function is to perform the indexing operation. 

For example, we can define a function quib that calculates the middle value of consecutive elements of an array:

In [20]:
r = iquib(np.array([0., 3., 2., 5., 8.]))
r_middle = (r[0:-1] + r[1:]) * 0.5
r_middle

r_middle = (r[0:-1] + r[1:]) * 0.5

In [21]:
r_middle.get_value()

array([1.5, 2.5, 3.5, 6.5])

Note that `r_middle` is defined functionaly; if its argument change it will get re-evaluated:

In [22]:
r[-1] = 13.
r_middle.get_value()

array([1.5, 2.5, 3.5, 9. ])

### Defining function quibs for user functions or for ״unsupported״ functions

Many *Python*, *NumPy* and *Matplotlib* functions are supported to work directly on quibs [Supported functions]]. Yet, not all functions are not currently supported to work directly on quib arguments. Similarly, any typical user function that expect normal, non-quib, arguments cannot work directly with quib arguments.
Therefore, *Quibbler* also provides a generic way to create function quibs that apply *any* Python function (whether supported or not), or any user-function, to the value of specfied quib arguments. This is done using the `q` function. The syntax `w = q(fun, *args, **kwards)` creates a new function-quib `w` whose function is to apply the function `fun` on the provided args and kwards arguments, while replacing any quib arguments with their respective values. 

For example, consider the Python function `str`. Applying `str(quib)` returns the string representation of the quib:

In [28]:
str(mean_z_sqr)

'mean_z_sqr = average(z_sqr)'

But, we may want instead to create a function quib that performs `str` on the *value* of its argument. To create an f-quib that performs the function `str` on the value of `mean_z_sqr`, we can use the `q` syntax:

In [17]:
str_mean_z_sqr = q(str, mean_z_sqr)
str_mean_z_sqr # this is a function-quib:

str_mean_z_sqr = str(mean_z_sqr)

In [18]:
str_mean_z_sqr.get_value() # the value of this function-quib is the string '3.5'

'3.5'

### Matplotlib functions too can work directly on quibs, creating "live" graphics

*Matplotlib* functions too can work directly with quib arguments, creating a *graphics quib*, which represents "live" graphics: graphics that automatically refreshes upon upstream changes. 

For example:

In [15]:
plt.plot(z_sqr, '-o', markersize=20)
plt.ylabel(str(z_sqr));

[[/images/graphics_refresh.gif]]

Note that unlike regular quibs which evaluate *lazily*, graphics quibs are evaluated *eagerly*, immediately upon creation, and are also recalculated immediately upon upstream changes, thereby enabling the above behavior.

WIP

In [24]:
n = q(len, r)
plt.plot(np.arange(n), r, 'x-', picker=True)
plt.plot(np.arange(n - 1) + 0.5, r_middle, 'or', picker=True)

plot(<AxesSubplot:>, arange(n - 1) + 0.5, r_middle, 'or', scalex=True, scaley=True, picker=True)

### Quibs can be combined with graphics functions, readily forming interactive GUIs.

Graphical functions too can be applied to quibs. Such graphics is bi-directionally linked to the quib: changes to upstream quibs refresh the graphics, and user interactions with the graphics can change upstream quibs (see also a separate chapter on [[Inverse assignments]]).

Consider, for example, the presentation of a graphics marker whose x-y coordinates are specified by quibs. Moving the graphics marker will change the values of x and y. Any quib graphics that depend on x or y will immediately refresh. 

In [22]:
# define and plot a curve:
curve_function = lambda v: 4 * v ** 2 - v ** 3
graph_xs = np.arange(0, 4, .2)
graph_ys = curve_function(graph_xs)
plt.figure(figsize=(4,3))
plt.plot(graph_xs, graph_ys, 'k')
plt.axis([0, 4, 0, 12])

# define x-y quibs:
point_x = iquib(3.)
point_y = q(curve_function, point_x)

# Plot the point, use picker=True to allow dragging
plt.plot(point_x, point_y, marker='o', markerfacecolor='c', 
         markersize=18, picker=True, pickradius=20)

# Define and plot text (this text will change when the marker is dragged):
xy_str = q("X={:.2f}, Y={:.2f}".format, point_x, point_y)
plt.text(point_x, point_y + .6, xy_str, 
         horizontalalignment="center",
         verticalalignment="bottom", fontsize=13);

[[/images/demo_gif/quibdemo_drag_on_curve.gif]]